# Deep Learning Final Project

Submitted by: Albert Khayat - 212747026, David Afik - 318267721

# Setup

### Imports

In [1]:
import torch

from os import path
from importlib import reload  		# for reloading modules
from collections import namedtuple	# for parameters

from model.MainModel import Painter
from data import ImagesDataset


### Configurations

In [2]:
DATA_PATH = "data\images"
SAVE_PATH = "results"

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Name of current CUDA device: {torch.cuda.get_device_name(torch.cuda.current_device())}")

else:
    device = torch.device('cpu')
    print('Running on CPU')

CUDA version: 12.1
Name of current CUDA device: NVIDIA GeForce RTX 3060 Laptop GPU


### Load Data

In [4]:
TRAIN_SIZE = 8500

BATCH_SIZE_TRAIN = 500
BATCH_SIZE_TEST = 800
SPLIT_SEED = None

# Get dataset
train_data, test_data = ImagesDataset.create_datasets(DATA_PATH, TRAIN_SIZE, SPLIT_SEED)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE_TRAIN, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE_TEST, shuffle=True, pin_memory=True)

# Train Model

In [7]:
reload(Painter)
# Training parameters
SAVE_MODEL = False		# If True, saves model every epoch
MODEL_NAME = "Model1"	# Used for save path

NUM_EPOCHS = 30

In [ ]:
# model parameters
Hyparams = namedtuple('Hyperparameters', ['lr_d','lr_g'])

hyparams = Hyparams(0.01, 0.01)
g_params = None
d_params = None

# create model
model = Painter(hyparams, g_params, d_params, MODEL_NAME)

print(f"Number of parameters: {model.count_parameters()}")

In [ ]:
if MODEL_NAME is not None:
	model = Painter(load_path=MODEL_NAME)

# Train model
for _ in range(NUM_EPOCHS):
	model.train_model(train_loader, test_loader, SAVE_MODEL)

	if SAVE_MODEL: # save backup
			model.save()

# Test Model

In [9]:
MODEL_PATH = None

if MODEL_PATH is not None:
    model = Painter(load_path=MODEL_PATH)
    
# test model

Test accuracy = 0.5116
